In [1]:
# Import required libraries
from openai import OpenAI
import pandas as pd

In [3]:
# Initialise OPen API key and instantiate an API client  
openai_api_key = "#########################################"
client = OpenAI(api_key=openai_api_key)

In [4]:
# Load datasets
nasdaq100 = pd.read_csv('nasdaq100.csv')
nasdaq100.head()

,symbol,name,headQuarter,dateFirstAdded,cik,founded
0,AAPL,Apple Inc.,"Cupertino, CA",NaN,320193,1976-04-01
1,ABNB,Airbnb,"San Francisco, CA",NaN,1559720,2008-08-01
2,ADBE,Adobe Inc.,"San Jose, CA",NaN,796343,1982-12-01
3,ADI,Analog Devices,"Wilmington, MA",NaN,6281,1965-01-01
4,ADP,ADP,"Roseland, NJ",NaN,8670,1949-01-01


In [5]:
nasdaq100_price_change = pd.read_csv('nasdaq100_price_change.csv')
nasdaq100_price_change.head()

,symbol,1D,5D,1M,3M,6M,ytd,1Y,3Y,5Y,10Y,max
0,AAPL,-1.7254,-8.30086,-6.20411,3.04200,15.64824,42.99992,8.479410,60.96299,245.42031,976.99441,139245.53954
1,ABNB,2.1617,-2.21919,9.88336,19.43286,19.64241,68.66902,23.640130,-1.04347,-1.04347,-1.04347,-1.04347
2,ADBE,0.5409,-1.77817,9.16191,52.04650,38.01522,57.22723,21.962060,17.83037,109.05718,1024.69214,251030.66399
3,ADI,0.9291,-4.03352,2.58486,3.65887,5.01602,17.02062,8.097350,63.42847,92.81874,286.77518,26012.63736
4,ADP,2.0589,2.35462,14.66581,16.40059,10.60546,5.53732,0.888943,81.76679,81.87224,248.40950,27613.11042


In [6]:
# Subset nasdaq100_price_change dataset to keep only colum of interest (ytd) and merge with nasdaq100 dataset
nasdaq100_price_change_subset = nasdaq100_price_change[['symbol','ytd']]
nasdaq100 = nasdaq100.merge(nasdaq100_price_change_subset, on='symbol', how='inner')

In [7]:
# Define a prompt to classify stocks by sector
prompt = """Classify company {company} into one of the following sectors: Technology, Consumer Cyclical, Industrials, Utilities, Healthcare, Communication, Energy, Consumer Defensive, Real Estate, or Financial. Answer only with the sector name.
"""

In [8]:
# Loop through each company and create a request to the completions endpoint. Store output in a new column "Sector"
for company in nasdaq100['name']:
    
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{ "role": "user", "content": prompt}],
    temperature = 0.0,
    )
    
    sector = response.choices[0].message.content
    nasdaq100.loc[nasdaq100["name"] == company, "Sector"] = sector

In [10]:
# Define a prompt to provide summary information about Nasdaq100 stock performance YTD, recommending the three best sectors and three or more companies per sector.
prompt = """Provide a summary of the Nasdaq100 year-to-date (ytd) stock performance based on Company data: {nasdaq100}. Recommend the three best sectors and at least three companies in each sector.
"""

response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{ "role": "user", "content": prompt}],
        temperature=0.0,
    )

stock_recommendations = response.choices[0].message.content
print(stock_recommendations)

The Nasdaq100 year-to-date stock performance has been strong, with a significant increase in value. The three best sectors based on this data are technology, healthcare, and consumer discretionary.

1. Technology Sector:
- Apple Inc. (AAPL)
- Microsoft Corporation (MSFT)
- Amazon.com Inc. (AMZN)

2. Healthcare Sector:
- Moderna Inc. (MRNA)
- Regeneron Pharmaceuticals Inc. (REGN)
- Vertex Pharmaceuticals Inc. (VRTX)

3. Consumer Discretionary Sector:
- Tesla Inc. (TSLA)
- Amazon.com Inc. (AMZN)
- Nike Inc. (NKE)

Overall, these sectors have shown strong performance in the Nasdaq100 year-to-date, with the mentioned companies leading the way in terms of stock performance. Investors may consider these sectors and companies for potential investment opportunities.
